# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|█████████████████████████████████████████████████████████████████████| 5/5 [02:14<00:00, 26.99s/it]


In [4]:
len(deals)

50

In [5]:
deals[44]

<Pumpkin-Shaped Lounge Chair for $127 + free shipping>

In [6]:
deals[44].describe()

'Title: Pumpkin-Shaped Lounge Chair for $127 + free shipping\nDetails: Apply coupon code "USAFF10" to cut an extra $10 off this already-cheap pumpkin-shaped chair. It\'s available at this price in six different colors.If you\'re a new customer, apply coupon code "NEWUS12OFF" instead of the coupon above to take an extra $12 off. Buy Now at AliExpress\nFeatures: \nURL: https://www.dealnews.com/Pumpkin-Shaped-Lounge-Chair-for-127-free-shipping/21741583.html?iref=rss-c196'

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: INIU 10,000mAh Portable Power Bank for $20 + free shipping
Details: It's $10 under list price. Buy Now at Best Buy
Features: built-in USB-C cable Model: P61L-P1
URL: https://www.dealnews.com/products/INIU/INIU-10-000-m-Ah-Portable-Power-Bank/489909.html?iref=rss-c142

Title: Certified Refurb Soundcore by Anker C30i Open-Ear Clip-On Earbuds for $42 + free shipping
De

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[1]

Deal(product_description='The Soundcore by Anker C30i Open-Ear Clip-On Earbuds are engineered for stunning sound quality with a secure and comfortable fit, making them ideal for active lifestyles. Featuring 12mm x 17mm drivers, these earbuds are also IPX4 water-resistant, providing confidence while exercising or commuting in various weather conditions. With an impressive playback time of up to 30 hours, they make an excellent choice for all-day listening.', price=42.0, url='https://www.dealnews.com/products/Soundcore-by-Anker/Soundcore-by-Anker-C30-i-Open-Ear-Clip-On-Earbuds/477620.html?iref=rss-c142')

In [14]:
from agents.scanner_agent import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='The INIU 10,000mAh Portable Power Bank features a built-in USB-C cable for easy charging on-the-go. This lightweight power bank is designed to provide reliable power for your devices, making it an essential accessory for travel and daily use. With its compact design, you can easily slip it into your bag or pocket without any hassle.', price=20.0, url='https://www.dealnews.com/products/INIU/INIU-10-000-m-Ah-Portable-Power-Bank/489909.html?iref=rss-c142'), Deal(product_description='The Soundcore by Anker C30i Open-Ear Clip-On Earbuds offer a secure fit with clip-on design and attachable ear grips for active users. With 12mm x 17mm drivers, these earbuds deliver excellent sound quality, making them perfect for extended listening sessions. Their IPX4 water resistance provides durability during workouts or outdoor activities, along with an impressive 30 hours of playtime.', price=42.0, url='https://www.dealnews.com/products/Soundcore-by-Anker/S